In [1]:
import pandas as pd
import numpy as np

In [2]:
# from transformers import LlamaModel, LlamaConfig
# from accelerate import init_empty_weights
# config = LlamaConfig()
# with init_empty_weights():
#     model = LlamaModel(config)

In [3]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer


# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)


# hf_model_repo = "meta-llama/Llama-2-7b-chat-hf"
hf_model_repo = "ilufy/meta-llama-2-7b-mk-physics-qna-finetuned-2600"
# hf_model_repo = "ilufy/meta-llama-2-7b-mk-physics-domain-tuned-2500"


# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)

# Load the model
model = AutoModelForCausalLM.from_pretrained(hf_model_repo,
                                             quantization_config=bnb_config,
                                             device_map="auto")

# prompt = "Question: According to E. T. Bell, what sparked Riemann's early interest in number theory?"
# prompt = "How many watts does the human brain use?"
#prompt = "Who was Einstein's girlfriend?"
#prompt = "What is the vision of Susumu Tachi regarding virtual and real reality?"
#prompt = "Question: Is there an unsolvable problem in Math?"
#prompt = "How can membranes like a beach ball induce resonance frequencies?"
#prompt = "Who is Michio Kaku, and what does he discuss in his book Hyperspace?"


use_transformers_pipeline = False

if use_transformers_pipeline:
    import transformers

    tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,  trust_remote_code=True)
    pipeline = transformers.pipeline(
        "text-generation", #"question-answering",
        model=hf_model_repo,
        trust_remote_code=True
    )

    sequences = pipeline(
        prompt,
        temperature=0.6,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )

    for seq in sequences:
        print(seq['generated_text'])



/home/aribra/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/aribra/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [59]:
# prompt = "Question: When was the concept of a nuclear-fired rocket briefly resurrected?"
# prompt =  "Question: What is the difference between a Type 1 Type 2 and Type 3 civilizations"
# prompt = "Question: What is the difference between a Type I Type II and Type III civilizations?"

# prompt = "Question: What is the current rate of production of antimatter?"
# is between one-billionth to ten-billionths of a gram per year.


# Type 0 civilization will reach Type I status.

# prompt = "Question: In which TV series did the concept of wormhole appear?"

# prompt = "Question: What happened to the Kaluza-Klein idea After the 1930s?"


# prompt = "Question: How did mystics contribute to the understanding of hyperspace?"

# prompt = "Question: What is the fundamental problem faced by physicists in unifying Einstein's theory of relativity with quantum field theory?"

# prompt = "Question: How was the Kaluza-Klein theory resurrected?"


# prompt = "Question: what is 'sticky, fat fingers' nanobots?"

# prompt = "Question: What are the dangers awaiting us in outer space?"

# prompt = "Question: What part of the brain is associated with body sensations and social emotions?"


# prompt = "Question: Explain about the insula part of the brain which is associated with body sensations and social emotions?"

# prompt = "Question: What is the main point that Coleman is trying to convey through his humorous writing style?"

# prompt = "Question: When did Einstein write a paper that \"proved\" black holes could never form?"

# prompt = "Question: What is the alternative name for Einstein-Rosen Bridge?"

# prompt = "Question: What does the text talk about?"

# prompt = "Question: What are the problems of the Standard Model?"

# prompt = "What are the various particles and forces within the Standard Model?"

# prompt = "How do the soap bubbles in the analogy represent universes?"

# prompt = "Question: What is the analogy of the bubbles?"

# prompt = "Question: What is meant by the UNIFICATION IN TEN DIMENSIONS?"

#prompt = "Question: What is the superstring theory according to the text?"

# prompt = "Question: What is the Standard Model compared to according to the text?"

# prompt = "QUESTION: Can time travel create problems in the timeline?"

# prompt = "Question: Who is the director of GlobalSecurity.org"

# prompt = "What is the grandfather paradox?"

# prompt = "Question: How do the apes treat humans in the advanced civilization?"

# prompt = "Question: What are the equations of Einstein mentiond in the text"

prompt = "Question: Who wrote down the field equation for electricity and magnetism."



# Generate response
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids
outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=200,
                         temperature=0.6)

# print("outputs: ", outputs)

result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Print the result
print(f"Generated response:\n{result}")


Generated response:
Question: Who wrote down the field equation for electricity and magnetism.


In [6]:
# free VRAM
del model
import gc
gc.collect()

torch.cuda.empty_cache()
